In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 17.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel
from google.colab import drive, files
import torch
import os
import pandas as pd
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_dir = "drive/My Drive/Colab Notebooks/"
file_name = "perplexity_corpus.csv"

In [ ]:
df = pd.read_csv(root_dir + file_name)

In [ ]:
def tokenize_seq(sent,tokenizer):
  return tokenizer('<sos>'+ sent + '<eos>')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def getPerplexity(row, model, tokenizer):
    para = row.full_paragraph
    idx = row.name
    n_rows = len(df)

    milestone = round(n_rows / 10)

    if idx % milestone == 0:
        print(f"\t{idx/milestone*10}% Complete")
    perps = []
    sentences = nltk.tokenize.sent_tokenize(para)
    for sent in sentences:
      enc_inputs = tokenizer.encode(sent, padding=True)
      enc_inputs = torch.tensor(enc_inputs)
      # enc_inputs['input_ids'] = torch.tensor(enc_inputs["input_ids"])
      # enc_inputs['attention_mask'] = torch.tensor(enc_inputs["attention_mask"])
      # enc_inputs['labels'] = enc_inputs['input_ids'].clone()

      with torch.no_grad():
          outputs = model(enc_inputs)
          logits = outputs.logits
          loss = outputs.loss

      sm = torch.nn.functional.softmax(logits, dim=1)

      word_probs = sm[torch.arange(sm.size(0)-1), enc_inputs[1:]]

      log_probs = torch.log(word_probs)

      perplexity = torch.exp(-torch.sum(log_probs)/len(enc_inputs))

      perps.append(perplexity)

    

    return np.mean(perps)

In [ ]:
def getPerplexity(row, model, tokenizer):
    para = row.full_paragraph
    idx = row.name
    n_rows = len(df)

    milestone = round(n_rows / 10)

    if idx % milestone == 0:
        print(f"\t{idx/milestone*10}% Complete")
    sentences = nltk.tokenize.sent_tokenize(para)
    for sent in sentences:
      enc_inputs = tokenizer.encode(sent, padding=True)
      enc_inputs = torch.tensor(enc_inputs)
      # enc_inputs['input_ids'] = torch.tensor(enc_inputs["input_ids"])
      # enc_inputs['attention_mask'] = torch.tensor(enc_inputs["attention_mask"])
      # enc_inputs['labels'] = enc_inputs['input_ids'].clone()

      with torch.no_grad():
          outputs = model(enc_inputs)
          logits = outputs.logits
          loss = outputs.loss

      sm = torch.nn.functional.softmax(logits, dim=1)

      word_probs = sm[torch.arange(sm.size(0)-1), enc_inputs[1:]]

      log_probs = torch.log(word_probs)

      perplexity = -torch.sum(log_probs)/len(enc_inputs)

      return torch.exp(perplexity)

In [ ]:
model_paths = []
for i,j in zip(['adjunct_island','adjunct_island','complexNP_island','complexNP_island','wh_island','wh_island'],['0','1','0','1','0','1']): 
  model_paths.append(root_dir+"GPT2_checkpoints/"+i+"/model_"+j+"_lr_.00000625_250")
print(model_paths)
results_df = df.copy()

['drive/My Drive/Colab Notebooks/GPT2_checkpoints/adjunct_island/model_0_lr_.00000625_250', 'drive/My Drive/Colab Notebooks/GPT2_checkpoints/adjunct_island/model_1_lr_.00000625_250', 'drive/My Drive/Colab Notebooks/GPT2_checkpoints/complexNP_island/model_0_lr_.00000625_250', 'drive/My Drive/Colab Notebooks/GPT2_checkpoints/complexNP_island/model_1_lr_.00000625_250', 'drive/My Drive/Colab Notebooks/GPT2_checkpoints/wh_island/model_0_lr_.00000625_250', 'drive/My Drive/Colab Notebooks/GPT2_checkpoints/wh_island/model_1_lr_.00000625_250', 'drive/My Drive/Colab Notebooks/GPT2_checkpoints/mix_construction/model_0_lr_.00000625_250', 'drive/My Drive/Colab Notebooks/GPT2_checkpoints/mix_construction/model_1_lr_.00000625_250']


In [ ]:
for mp in model_paths:
  print(f"Evaluating {mp}...")
  tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<sos>', eos_token='<eos>', pad_token='<pad>')
  model = GPT2LMHeadModel.from_pretrained(mp)
  pps = df.apply(getPerplexity, axis=1, model=model, tokenizer=tokenizer)
  results_df.loc[:, "pps"] = pps
  print(results_df['pps'].mean())
  print(results_df['pps'].std()/torch.sqrt(torch.tensor(447)))

Evaluating drive/My Drive/Colab Notebooks/GPT2_checkpoints/adjunct_island/model_0_lr_.00000625_250...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


	0.0% Complete
	10.0% Complete
	20.0% Complete
	30.0% Complete
	40.0% Complete
	50.0% Complete
	60.0% Complete
	70.0% Complete
	80.0% Complete
	90.0% Complete
2.556870163922088e+27
tensor(inf)
Evaluating drive/My Drive/Colab Notebooks/GPT2_checkpoints/adjunct_island/model_1_lr_.00000625_250...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


	0.0% Complete
	10.0% Complete
	20.0% Complete
	30.0% Complete
	40.0% Complete
	50.0% Complete
	60.0% Complete
	70.0% Complete
	80.0% Complete
	90.0% Complete


...

	0.0% Complete
	10.0% Complete
	20.0% Complete
	30.0% Complete
	40.0% Complete
	50.0% Complete
	60.0% Complete
	70.0% Complete
	80.0% Complete
	90.0% Complete


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


9236.550894854587
tensor(4030.4827)
Evaluating drive/My Drive/Colab Notebooks/GPT2_checkpoints/wh_island/model_1_lr_.00000625_250...
	0.0% Complete
	10.0% Complete
	20.0% Complete
	30.0% Complete
	40.0% Complete
	50.0% Complete
	60.0% Complete
	70.0% Complete
	80.0% Complete
	90.0% Complete
6764273.109619686
tensor(1131022.3750)
Evaluating drive/My Drive/Colab Notebooks/GPT2_checkpoints/mix_construction/model_0_lr_.00000625_250...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


OSError: ignored

In [ ]:
model_paths_ = ['gpt2']
for i,j in zip(['mix_construction','mix_construction'],['0','1']): 
  model_paths_.append(root_dir+"GPT2_checkpoints/"+i+"/model_"+j+"_lr_.00000625_65")
results_df = df.copy()
for mp in model_paths_:
  print(f"Evaluating {mp}...")
  tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<sos>', eos_token='<eos>', pad_token='<pad>')
  model = GPT2LMHeadModel.from_pretrained(mp)
  pps = df.apply(getPerplexity, axis=1, model=model, tokenizer=tokenizer)
  results_df.loc[:, "pps"] = pps
  print(results_df['pps'].mean())
  print(results_df['pps'].std()/torch.sqrt(torch.tensor(447)))

Evaluating gpt2...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


	0.0% Complete
	10.0% Complete
	20.0% Complete
	30.0% Complete
	40.0% Complete
	50.0% Complete
	60.0% Complete
	70.0% Complete
	80.0% Complete
	90.0% Complete
120.081604
tensor(3.6052)
Evaluating drive/My Drive/Colab Notebooks/GPT2_checkpoints/mix_construction/model_0_lr_.00000625_65...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


	0.0% Complete
	10.0% Complete
	20.0% Complete
	30.0% Complete
	40.0% Complete
	50.0% Complete
	60.0% Complete
	70.0% Complete
	80.0% Complete
	90.0% Complete
inf
tensor(nan)
Evaluating drive/My Drive/Colab Notebooks/GPT2_checkpoints/mix_construction/model_1_lr_.00000625_65...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


	0.0% Complete
	10.0% Complete
	20.0% Complete
	30.0% Complete
	40.0% Complete
	50.0% Complete
	60.0% Complete
	70.0% Complete
	80.0% Complete
	90.0% Complete
inf
tensor(nan)


In [ ]:
display(results_df)
print(type(results_df))
print(results_df[' pps'].mean())
print(results_df[' pps'].std()/torch.sqrt(torch.tensor(447)))
#results_df.to_csv('.csv')
#files.download('.csv')

,id,full_paragraph,pps
0,cj50_0,"Unfortunately , however , and for reasons to ...",tensor(33.3928)
1,cf08_37,"One such wife , Dr. Linden says , became disg...",tensor(59.2077)
2,cl06_11,"The kick came , sudden and vicious but short ...",tensor(35.0073)
3,cl06_28,In the garage he checked the Jeep for signs o...,tensor(39.0804)
4,cf30_4,Dartmouth students enjoy other unusual divers...,tensor(95.3046)
...,...,...,...
442,ch21_11,A substantial increase is estimated in the co...,tensor(58.1177)
443,ch21_24,These examples underscore the importance of e...,tensor(39.7950)
444,ch19_17,The first year's projects should also be spre...,tensor(45.3037)
445,ch19_23,The already appropriated funds within the dis...,tensor(40.3886)


<class 'pandas.core.frame.DataFrame'>
70.87383773769575
tensor(2.0163)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>